## 1. Setup and Configuration

In [19]:
import chromadb
import pickle
import numpy as np
from tqdm import tqdm
from pathlib import Path
import json

def load_config(path: str):
    """Loads a JSON config file."""
    with open(path, 'r') as f:
        return json.load(f)

config = load_config('../frontend/config.json')

In [20]:
# --- IMPORTANT: CONFIGURE THIS PATH ---
# Point this to the latest run directory created by the training script.
# Example: ARTIFACTS_PATH = "../backend/artifacts/run_20240101_120000"

ARTIFACTS_PATH = config['ARTIFACTS_PATH']
print("Loading artifacts from: ", ARTIFACTS_PATH)
CHROMA_STORE_PATH = "./chroma_store"
COLLECTION_NAME = "docs"

Loading artifacts from:  artifacts/run-20250619_232020


## 2. Load Artifacts

In [21]:
artifacts = Path('../' + ARTIFACTS_PATH)

if not artifacts.exists():
    raise FileNotFoundError(f"Artifacts directory not found at {ARTIFACTS_PATH}. Please run the backend training script first.")

# Load documents
with open(artifacts / 'documents.pkl', 'rb') as f:
    documents = pickle.load(f)

# Load embeddings
embeddings = np.load(artifacts / 'document_embeddings.npy')

print(f"Loaded {len(documents)} documents and {embeddings.shape[0]} embeddings.")

Loaded 185828 documents and 185828 embeddings.


## 3. Setup ChromaDB and Index Documents

In [22]:
client = chromadb.PersistentClient(path=CHROMA_STORE_PATH)

# Delete the collection if it already exists to ensure a fresh start
if COLLECTION_NAME in [c.name for c in client.list_collections()]:
    print(f"Collection '{COLLECTION_NAME}' already exists. Deleting it.")
    client.delete_collection(name=COLLECTION_NAME)

collection = client.get_or_create_collection(name=COLLECTION_NAME)
print(f"Created new collection: '{COLLECTION_NAME}'")

Collection 'docs' already exists. Deleting it.


InternalError: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [11]:
batch_size = 500
num_docs = len(documents)

for i in tqdm(range(0, num_docs, batch_size)):
    batch_end = min(i + batch_size, num_docs)
    
    batch_docs = documents[i:batch_end]
    batch_embeddings = embeddings[i:batch_end]
    batch_ids = [f"doc_{j}" for j in range(i, batch_end)]
    
    collection.add(
        ids=batch_ids,
        documents=batch_docs,
        embeddings=batch_embeddings.tolist()
    )

print(f"\n✅ Successfully indexed {collection.count()} documents into ChromaDB.")

100%|██████████| 372/372 [02:07<00:00,  2.93it/s]


✅ Successfully indexed 185828 documents into ChromaDB.


## 4. (Optional) Test Query

In [18]:
# This is a simple test and does not use the trained query encoder.
# It finds documents similar to the embedding of another document.
results = collection.query(
    query_embeddings=[embeddings[0].tolist()],
    n_results=5
)

print("Querying with the first document as an example:")
print(documents[0])
print("\nResults:")
for doc in results['documents'][0]:
    print(f"  - {doc[:100]}...")

Querying with the first document as an example:
For example-The place value of 8 in 80 is tens, while the place value of 5 and 6 in 576 are hundreds and ones. In other words, in the number 80, the position of 8 is tens.t is by using these ten digits that numbers are formed. The value of ten digit in a number is determined by its place in the number and is know as the place value. For example, to write the numeral for two thousand four hundred and eighty seven, we write 2487.

Results:
